# Want to run this code for yourself?
You can find the interactive ipython notebook where you can run all the steps listed here at

https://mybinder.org/v2/gh/NanoNets/tutorials/master?filepath=object_detection/notebooks/object_detection.ipynb

PS: it will take a couple of minutes for the mybinder instance to boot up and be ready for use.

In [2]:
import json
import os
import requests 

In [1]:
# Helper methods for launching an MLC model

BASE_MLC_URL = "http://app.nanonets.com/api/v2/MultiLabelClassification/"


class MLCUtils:

    def __init__(self, auth_key, base_url=BASE_MLC_URL):
        self.auth_key = auth_key
        self.base_url = base_url

    def create_new_model(self, categories):
        """function to create a new Multi Label Classification model for training
         
        Parameters
        ----------
        categories: List[str]
            List of labels you want to predict
        
        Returns
        -------
        A tuple of (model_id, model_type, categories)
            model_id unique identifier for the model created
            model_type what type of model was created, "object_detection"
            categories what categories the model will detect
        """

        url = self.base_url + "Model/"
        data = json.dumps({'categories' : categories})

        response = requests.request("POST", url, auth=requests.auth.HTTPBasicAuth(self.auth_key, ''), data=data)
        result = json.loads(response.text)
        model_id, model_type, categories = (result["model_id"], result["model_type"], result["categories"])
        return model_id, model_type, categories

    def get_model_info(self, model_id):
        url = "{}Model/{}".format(self.base_url, model_id)
        response = requests.request("GET", url, auth=requests.auth.HTTPBasicAuth(self.auth_key, ''))
        result = json.loads(response.text)
        model_id, model_type, categories, state = (result["model_id"], result["model_type"], result["categories"], result["state"])
        return model_id, model_type, categories, state


    def upload_images(self, model_id, images_label_list):
        """function to upload data for a MLC model that has been created
        The function will automatically upload the data in batches of 50 images so
        you can provide all the image and label tuples in one call.
        
        Parameters
        ----------
        model_id: str
            model id of the model for which data is being uploaded generated by calling the create_model method
        image_label_list: List[(image_path, List[str])]
            a list of image to label tuples that will be uploaded to the model
            each element of the list is a tuple of (image_path, List[labels])
            image_path -> path to where image is found on local machine
            List[labels] -> all the labels that should be associated with the image

        Returns
        -------
        None
        """
        url = "{}Model/{}/UploadFiles/".format(self.base_url, model_id)
        n = len(images_label_list)
        image_uploaded = 0
        all_responses = []
        while len(images_label_list) > 0:
            batch_images, all_images = images_label_list[:50], images_label_list[50:]
            multiple_files = []
            multiple_data = []

            for (image, labels) in batch_images:
                _image_dir, image_name = os.path.split(image)
                image_data = {'filename': image_name, "categories": labels}
                multiple_data.append(image_data)
                multiple_files.append(('files', (image_name, open(image, 'rb'), 'image/jpeg')))
            multiple_files.append(('data', ('', json.dumps(multiple_data))))
            response = requests.post(url, auth= requests.auth.HTTPBasicAuth(self.auth_key, ''), files=multiple_files)
            all_responses.append(response)
            
            image_uploaded += len(batch_images)
        
            if len(all_images) > 0:
                print("%d of %d images has been uploaded, uploading next batch...."%(image_uploaded, n))
            else:
                print("%d of %d images has been uploaded, Done uploading"%(image_uploaded, n))
        return all_responses

    def train_model(self, model_id):
        """launch training for mlc model

        Parameters
        ----------
        model_id: str
            model_id for model whose training should be initiated

        Returns
        -------
        None
        """
        url = '%sModel/%s/Train/' % (self.base_url, model_id)
        response = requests.request('POST', url, auth=requests.auth.HTTPBasicAuth(self.auth_key, ''))
        print(json.loads(response.text))

In [19]:
AUTH_KEY = "<AUTH_KEY_FROM_NANONETS_APP>" ## can be foung https://app.nanonets.com/#/keys

# path to where the images and annotations for the MLC model are stored.
# The annotation file for an image for MLC should have all labels associated with the given image on a new line
data_directory = "../data/"
image_folder = os.path.join(data_directory, "ImageSets")
annotations_folder = os.path.join(data_directory, "Annotations")
images = os.listdir(image_folder)
CATEGORIES = set()
image_label_list = {}

for image in images:
    if not image or image == ".DS_Store":
        continue
    image_name, extension = os.path.splitext(image)
    xml_file = os.path.join(annotations_folder, image_name + ".txt")
    labels = []
    with open(xml_file, "r") as f:
        labels = f.readlines()
    labels = [label.strip() for label in labels]
    CATEGORIES.update(labels)
    image_label_list[os.path.join(image_folder, image)] = labels

In [23]:
mlc_utils = MLCUtils(auth_key=AUTH_KEY)

# create a new model to upload data to and train
model_id, model_type, categories = mlc_utils.create_new_model(categories=list(CATEGORIES))

# upload training data to the model created
mlc_utils.upload_images(model_id, image_label_list)

# launch training on Nanonets Server
mlc_utils.train_model(model_id)

KeyError: 'model_id'